In [1]:
import os
import numpy as np
import pandas as pd
import difflib
from skimage import io
import matplotlib.pyplot as plt
import re
from PIL import Image
from joblib import Parallel, delayed
from tqdm.notebook import tqdm
from DataManager import DataManager


In [2]:
dm = DataManager()

In [3]:
dm.cell_line_df

,name,mutationDetailed,mutation,startPlate
0,02-006,AKT1-/-&AKT2-/-,AKT1/2,1.0
1,02-031,MEK2-/-,MEK2,5.0
2,02-004,AKT1-/-,AKT1,9.0
3,104-009,CTNNB1 mt-/wt+,CTNNB1 wt,13.0
4,PAR007,PARENTAL007,HCT116 P2,17.0
5,104-001,P53-/-,P53,21.0
6,104-004,PTEN-/-,PTEN,25.0
7,104-007,PI3KCA mt-/wt+,PI3KCA wt,29.0
8,104-008,KRAS mt-/wt+,KRAS wt,33.0
9,02-008,BAX-/-,BAX,37.0


In [4]:
dm.drug_df

,PlateName,Well,Content,compoundID_384,compoundID,mol.weight.Structure,Name,SecName,Class,Enzyme,Action,Selectivity,Description,GeneID,Selectivity_updated
0,P1,A03,sample,78875,78875,"195,22001",DL-alpha-Methyl-p-tyrosine,4-Hydroxy-alpha-methylphenylalanine,Neurotransmission,Enzyme,Inhibitor,Tyrosine hydroxylase,Inhibitor of catecholamine synthesis and tyros...,78875,Tyrosine hydroxylase
1,P1,A04,sample,78955,78955,"163,19638",N-Acetyl-L-Cysteine,,Glutamate,,Antagonist,,Amino acid analog that partially improves neur...,78955,AA-analog
2,P1,A05,sample,78876,78876,"202,25818","6-Methoxy-1,2,3,4-tetrahydro-9H-pyrido[3,4b] i...",,Neurotransmission,Enzyme,Inhibitor,MAO,MAO inhibitor,78876,MAO
3,P1,A06,sample,78956,78956,"131,17601",6-Aminohexanoic acid,6-Aminocaproic acid; EACA,Immune System,,Inhibitor,Blood Clotting,"Promotes rapid dissociation of plasmin, thereb...",78956,Blood Clotting
4,P1,A07,sample,78877,78877,"59,06825",Acetamide,Amide C2,Biochemistry,Enzyme,Inhibitor,Carbonic anhydrase,Carbonic anhydrase inhibitor,78877,Carbonic anhydrase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,P4,P18,sample,80152,80152,"522,6251","( R)-(+)-WIN 55,212-2 mesylate","(R)-(+)-[2,3-Dihydro-5-methyl-3[(morpholinyl)m...",Cannabinoid,,Agonist,,High affinity cannabinoid receptor agonist,80152,NA
1368,P4,P19,sample,80073,80073,"423,34072","N,N,N-trimethyl-1-(4-trans-stilbenoxy)-2-propy...",F3,Cholinergic,,Antagonist,Nicotinic,Nicotinic acetylcholine receptor antagonist,80073,acetylcholine receptor
1369,P4,P20,sample,80153,80153,"428,49511",Xanthine amine congener,8-[4-[[[[(2-Aminoethyl)amino]carbonyl]methyl]o...,Adenosine,,Antagonist,A1,A1 Adenosine receptor antagonist,80153,adenosine receptor
1370,P4,P21,sample,80074,80074,"212,17589",1-[2-(Trifluoromethyl)phenyl]imidazole,TRIM,Nitric Oxide,Enzyme,Inhibitor,NOS,Potent nitric oxide synthase (NOS) inhibitor,80074,NOS


In [5]:
raws_dir = "/media/gvisona/GioData/idr0017/raw_images"
processed_dir = "/media/gvisona/GioData/idr0017/processed_images"

In [ ]:
counter = 0
with open("already_processed.txt", "w") as f:
    pass

for cl_folder_name in os.listdir(raws_dir):
    if not os.path.isdir(os.path.join(raws_dir, cl_folder_name)):
        continue

    name = cl_folder_name.replace(".", "/")
    name = name.replace("AND", "&")
    cl_name, plate_rep = name.split("_LOPAC_")
    cl_name = difflib.get_close_matches(cl_name, dm.cell_line_df["mutationDetailed"].values)[0]
    cl_index = dm.cell_line_df[(dm.cell_line_df["mutationDetailed"] == cl_name)].index[0]
    
    m = re.search(r'Plate_(\d)_Replicate_(\d)', plate_rep)
    plate, replicate = m.group(1), m.group(2)
    plate_name = "P"+str(plate)
    
    save_dir = os.path.join(processed_dir, "CL_{}-R_{}".format(cl_index, replicate))
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    files_list = [f for f in os.listdir(os.path.join(raws_dir, cl_folder_name)) if ".tif" in f and "Cy3" in f]
    
    drug_id = []
    idx = -1
    while not len(drug_id):
        idx += 1
        img_name = files_list[idx]
        mi = re.search(r'([a-zA-Z]) - (\d+)\(fld (\d) wv (.+)\).tif', img_name)
        well_letter = mi.group(1)
        well_number = mi.group(2)
        field = mi.group(3)
        well_name = well_letter + str(well_number) if len(str(well_number)) == 2 else well_letter + "0" + str(well_number)
        drug_id = dm.drug_df[(dm.drug_df["PlateName"] == plate_name) & (dm.drug_df["Well"] == well_name)]
    drug_id = drug_id.index[0]
    if os.path.exists(os.path.join(save_dir, "{}-{}-{}-{}.png".format(drug_id, cl_index, replicate, field))):
        with open("already_processed.txt", "a") as f:
            f.write(cl_folder_name + "\n")
        continue
    
    
    def process_image(img_name):
        mi = re.search(r'([a-zA-Z]) - (\d+)\(fld (\d) wv (.+)\).tif', img_name)
        well_letter = mi.group(1)
        well_number = mi.group(2)
        field = mi.group(3)
        channel = mi.group(4)
        second_channel_name = "{} - {}(fld {} wv {}).tif".format(well_letter, well_number, field, "DAPI - DAPI")

        well_name = well_letter + str(well_number) if len(str(well_number)) == 2 else well_letter + "0" + str(well_number)
        drug_id = dm.drug_df[(dm.drug_df["PlateName"] == plate_name) & (dm.drug_df["Well"] == well_name)]
        if len(drug_id):
            drug_id = drug_id.index[0]

            img_channel1 = io.imread(os.path.join(raws_dir, cl_folder_name, img_name))
            img_channel1 = (img_channel1-np.min(img_channel1))
            img_channel1 = img_channel1/np.max(img_channel1)
            img_channel1 = img_channel1*255

            img_channel2 = io.imread(os.path.join(raws_dir, cl_folder_name, second_channel_name))
            img_channel2 = (img_channel2-np.min(img_channel2))
            img_channel2 = img_channel2/np.max(img_channel2)
            img_channel2 = img_channel2*255
            stacked_image = np.concatenate((img_channel1[:,:, None], img_channel2[:,:, None], np.zeros((2048, 2048, 1))), axis=2).astype(np.uint8)
            io.imsave(os.path.join(save_dir, "{}-{}-{}-{}.png".format(drug_id, cl_index, replicate, field)), stacked_image)
    
    
    Parallel(n_jobs=4)(delayed(process_image)(img_name) 
                       for img_name in tqdm(files_list)) 
                       
    counter += 1
    if counter > 4:
        break